In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, CSVLogger

from loader import load_20news
from custom_layer import MonteCarloLRF, SeparableMonteCarloLRF, SeparableMonteCarloMaxPoolingV2

import numpy as np
import sklearn as sk
import pickle

Using TensorFlow backend.


In [3]:
top_words=10000
sparse=False
remove_short_documents=True
notebook = 'mcNet_top10k_temptative_7'
batch_size = 16

In [4]:
(input_shape, nb_classes), (X_train, X_test, Y_train, Y_test), graph_data = \
    load_20news(data_home='data', top_words=top_words, sparse=sparse,
                remove_short_documents=remove_short_documents, verbose=False)

In [5]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
num_classes = Y_train.shape[1]

In [6]:
print(X_train.shape)
print(X_test.shape)
print(batch_size, num_classes)

(10168, 10000, 1)
(7071, 10000, 1)
16 20


In [7]:
%%time
#Process next cell only once
path = os.path.join('probabilities_' + 
                    'top' + str(top_words) +
                    '_sparse' + str(sparse) +
                    '_removeShorts' + str(remove_short_documents) +
                    '.pkl')
if os.path.isfile(path):
  probabilities = pickle.load(open(path, "rb"))

CPU times: user 5.34 s, sys: 458 ms, total: 5.8 s
Wall time: 5.8 s


In [8]:
if not(os.path.isfile(path)):
  METRIC = 'cosine'#'euclidean'
  distances = sk.metrics.pairwise.pairwise_distances(graph_data, metric=METRIC, n_jobs=-2)

  # enforce exact zero
  for k in xrange(distances.shape[0]):
    distances[k,k] = 0.

  # max normalize
  #distances /= distances.max()
  distances /= distances.max(axis=1).reshape((distances.shape[0], 1))

  # use tricube kernel (becaause of flatness around 0)
  probabilities = (1. - np.abs(distances) ** 3) ** 3

  # remove auto connections (which are taken anyway in LRF)
  for k in xrange(probabilities.shape[0]):
    probabilities[k,k] = 0.

  # normalize proba
  probabilities /= np.sum(probabilities, axis=1).reshape((probabilities.shape[0], 1))
  
  # pickled for later use
  pickle.dump(probabilities, open(path,"wb"))  

In [9]:
model = Sequential()
model.add(MonteCarloLRF(probabilities, LRF_size=16, filters=32, activation='relu',
                       input_shape=(X_train.shape[1], X_train.shape[2])))
#model.add(Conv1D(1, kernel_size=1, activation='relu',
#                 padding='same',kernel_initializer='he_uniform'))
#model.add(SeparableMonteCarloMaxPoolingV2(LRF_size=4, new_size=2500))
#model.add(Dropout(0.2))
model.add(Flatten())
#model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
monte_carlo_lrf_1 (MonteCarl (None, 10000, 32)         544       
_________________________________________________________________
flatten_1 (Flatten)          (None, 320000)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 320000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                6400020   
Total params: 6,400,564
Trainable params: 6,400,564
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stopper = EarlyStopping(min_delta=0.001, patience=2)
csv = CSVLogger(notebook + '_log.csv')

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    callbacks=[early_stopper, csv],
                    validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 10168 samples, validate on 7071 samples
Epoch 1/20
10168/10168 [==============================] - 100s - loss: 1.9404 - acc: 0.5450 - val_loss: 1.2843 - val_acc: 0.6749
Epoch 2/20
10168/10168 [==============================] - 99s - loss: 0.4893 - acc: 0.8937 - val_loss: 1.1079 - val_acc: 0.7073
Epoch 3/20
10168/10168 [==============================] - 99s - loss: 0.1770 - acc: 0.9693 - val_loss: 1.1709 - val_acc: 0.7057
Epoch 4/20
10168/10168 [==============================] - 99s - loss: 0.0747 - acc: 0.9910 - val_loss: 1.2432 - val_acc: 0.6981
Epoch 5/20
10168/10168 [==============================] - 99s - loss: 0.0382 - acc: 0.9956 - val_loss: 1.3568 - val_acc: 0.6901
Test loss: 1.35683490778
Test accuracy: 0.690142836847


In [11]:
%%bash
tail mcNet_top10k_temptative_7_log.csv

epoch,acc,loss,val_acc,val_loss
0,0.5450432730133753,1.9404424250266572,0.674869184159538,1.2842907058587423
1,0.89368607395751376,0.48927312082282021,0.7072549853107748,1.1079141713658818
2,0.96931549960660901,0.17695935824658812,0.70569933546498143,1.1708667914207544
3,0.9909520062942565,0.074665257441983462,0.69806250899907907,1.2431898087730942
4,0.9955743509047994,0.038191223081075282,0.69014283710820157,1.3568349116560998
